In [21]:
import requests 
import pandas as pd
import time
import json

# alpha vantage API
API_KEY = "31HRF2A9SFS01WQV"
BASE_URL = "https://www.alphavantage.co/query?function="

CURRENCY_PAIRS = ["EURUSD", "GBPUSD", "AUDUSD", "USDJPY", "USDCHF", "USDCAD"]

# 6 pairs: 4 API requests for each 💀
# 5 requests per minute limit; 25 requests per day limit (with free AlphaVantage account)

#### Function to get OHLC data

In [22]:
def get_ohlc_data(symbol, api_key):
    
    # https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=EUR&to_symbol=USD&apikey=demo
    url = BASE_URL + "FX_DAILY" + "&from_symbol=" + symbol[:3] + "&to_symbol=" + symbol[3:] + "&output_size=full" + "&apikey=" + api_key
    response = requests.get(url) 
    data = response.json()
    
    if 'Time Series FX (Daily)' in data:
        ohlc = pd.DataFrame(data["Time Series FX (Daily)"]).T
        ohlc.columns = ['open','high','low','close']
        ohlc.index = pd.to_datetime(ohlc.index)
        ohlc = ohlc.astype(float)
        return ohlc
    else:
        print(f"Error fetching {symbol[:3]}/{symbol[3:]} data.")

#### Function to get data from technical indicators (SMA, RSI)

In [23]:
def get_tech_data(symbol, indicator, api_key, time_period=200, series_type='close', interval='daily'):
    
    # https://www.alphavantage.co/query?function=RSI&symbol=USDEUR&interval=weekly&time_period=10&series_type=open&apikey=demo
    url = BASE_URL + indicator + "&symbol=" + symbol + "&interval=" + interval + "&time_period=" + str(time_period) + "&series_type=" + series_type + "&apikey=" + api_key
        
    response = requests.get(url)
    data = response.json()
    
    if 'Technical Analysis: ' + indicator in data:
        indicator_df = pd.DataFrame(data['Technical Analysis: ' + indicator]).T
        indicator_df.index = pd.to_datetime(indicator_df.index.astype(str), format='ISO8601')
        indicator_df = indicator_df.astype(float)
        return indicator_df
    else:
        print(f"Error fetching {indicator} indicator for {symbol[:3]}/{symbol[3:]}.")

#### Function to get STOCH data

In [24]:
def get_stoch_data(symbol, api_key, slowk_period=14, slowd_period=3, slowkmatype=0, slowdmatype=0):

    params = {
        'function': 'STOCH',
        'symbol': symbol,
        'interval': 'daily',
        'apikey': api_key,
        'slowkperiod': slowk_period,
        'slowdperiod': slowd_period,
        'slowkmatype': slowkmatype,
        'slowdmatype': slowdmatype
    }
    response = requests.get(BASE_URL, params=params)
    data = response.json()

    if 'Technical Analysis: STOCH' in data:
        stoch_df = pd.DataFrame(data['Technical Analysis: STOCH']).T
        stoch_df.index = pd.to_datetime(stoch_df.index.astype(str), format='ISO8601')
        stoch_df = stoch_df.rename(columns={'SlowK': 'SLOW_K', 'SlowD':'SLOW_D'})
        stoch_df = stoch_df.astype(float)
        return stoch_df
    else:
        print(f"Error fetching STOCH indicator for {symbol[:3]}/{symbol[3:]}.")

#### Collect data for each currency pair

In [25]:
all_data = {}

for symbol in CURRENCY_PAIRS:
    print(f"Fetching {symbol[:3]}/{symbol[3:]} data...")

    """OHLC"""
    # daily ohlc for last 200 days
    ohlc_data = get_ohlc_data(symbol, API_KEY)

    time.sleep(12)
    
    """Technical Indicators"""
    # simple moving averages for last 200 days 
    sma_200_data = get_tech_data(symbol, 'SMA', API_KEY, time_period=200)
    sma_200_data.columns = ['SMA_200']

    time.sleep(12)

    # relative strength index (RSI) for last 200 days ---> above 70 = overbought, below 30 = oversold
    rsi_data = get_tech_data(symbol, 'RSI', API_KEY, time_period=14)
    rsi_data.columns = ['RSI']

    time.sleep(12)

    # slow stochastic Oscillator ---> >80 = overbought, <20 = oversold
    stoch_data = get_stoch_data(symbol, API_KEY)
    
    combined_data = pd.concat([ohlc_data, sma_200_data, rsi_data, stoch_data], axis=1)
    combined_data = combined_data.dropna()

    all_data[symbol] = combined_data

    print(f"\n{symbol[:3]}/{symbol[3:]} Combined Data:")
    print(combined_data.head())
    
    # 5 API Requests per minute limit
    time.sleep(12)

Fetching EUR/USD data...

EUR/USD Combined Data:
               open     high      low    close  SMA_200      RSI   SLOW_K  \
2024-05-30  1.08354  1.08826  1.08116  1.08354   1.0789  54.2938  53.4004   
2024-06-02  1.08530  1.08909  1.08280  1.08530   1.0790  56.6113  53.1580   
2024-06-03  1.09074  1.09182  1.08607  1.09074   1.0790  62.8586  55.4196   
2024-06-04  1.08819  1.08899  1.08610  1.08819   1.0790  58.5917  53.7812   
2024-06-05  1.08745  1.08987  1.08637  1.08745   1.0790  57.3857  52.4433   

             SLOW_D  
2024-05-30  55.6166  
2024-06-02  53.5314  
2024-06-03  53.9927  
2024-06-04  54.1196  
2024-06-05  53.8814  
Fetching GBP/USD data...

GBP/USD Combined Data:
               open     high      low    close  SMA_200      RSI   SLOW_K  \
2024-05-30  1.27303  1.27655  1.27016  1.27304   1.2542  60.3456  63.4913   
2024-06-02  1.27441  1.27939  1.26955  1.27437   1.2543  61.5854  61.6417   
2024-06-03  1.28102  1.28172  1.27444  1.28102   1.2544  67.1151  64.0303   

#### Save data to CSVs

In [27]:
import os

downloads_path = '/Users/ytpatel3/Downloads'

for symbol, data in all_data.items():
    file_path = os.path.join(downloads_path, f'{symbol[:3]}_{symbol[3:]}_forex_data.csv')
    data.to_csv(file_path)